In [1]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=4, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2025-10-07 19:02:09.512 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\mwien\.cache\mads_datasets\gestures
100%|██████████| 651/651 [00:06<00:00, 101.30it/s]


In [3]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))

In [4]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [5]:
loss_fn = torch.nn.CrossEntropyLoss()

In [6]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

using cpu


Set up the settings for the trainer and the different types of logging you want

In [7]:
settings = TrainerSettings(
    epochs=16, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": True, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 16
metrics: [Accuracy]
logdir: gestures
train_steps: 650
valid_steps: 162
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': True, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [8]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [ ]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)
    
n_repeats = 3

for repeat in range(n_repeats):
    print(f"Repeat {repeat+1}/{n_repeats}")
    
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", f"{repeat}_GRU_16epochs_0.5drop_128hidsize_4batch")
        mlflow.set_tag("dev", "Marcello")
        # Log hyperparameters to MLflow

        mlflow.log_param("epochs", settings.epochs)

        mlflow.log_param("learning_rate", settings.optimizer_kwargs.get("lr", None))
        
        config = ModelConfig(
            input_size=3,
            hidden_size=128,
            num_layers=1,
            output_size=20,
            dropout=0.5,
        )

        model = GRUmodel(
            config=config,
        )

        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
        trainer.loop()

        if not settings.earlystop_kwargs["save"]:
            tag = datetime.now().strftime("%Y%m%d-%H%M-")
            modelpath = modeldir / (tag + "model.pt")
            torch.save(model, modelpath)

2025/10/07 19:02:45 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/07 19:02:45 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


Repeat 1/3


c:\Master Applied Data Science\Year 2\Semester 3 (Deep Learning & Model Deployment)\Portfolio-Marcello-Wienhoven\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
2025-10-07 19:02:46.357 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251007-190246
2025-10-07 19:02:48.685 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 650/650 [00:07<00:00, 90.37it/s]
2025-10-07 19:02:56.704 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.0589 test 1.3433 metric ['0.5278']
2025-10-07 19:02:56.705 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.3433 --> 1.3433).Saving gestures\20251007-190246\checkpoint.pt ...
100%|██████████| 650/650 [00:07<00:00, 89.80it/s]
2025

Repeat 2/3


100%|██████████| 650/650 [00:09<00:00, 65.05it/s]
2025-10-07 19:05:28.495 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.9893 test 1.3205 metric ['0.5432']
2025-10-07 19:05:28.497 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.3205 --> 1.3205).Saving gestures\20251007-190517\checkpoint.pt ...
100%|██████████| 650/650 [00:07<00:00, 82.68it/s]
2025-10-07 19:05:36.994 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.8707 test 0.4874 metric ['0.8704']
2025-10-07 19:05:36.995 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.3205 --> 0.4874).Saving gestures\20251007-190517\checkpoint.pt ...
100%|██████████| 650/650 [00:10<00:00, 62.55it/s]
2025-10-07 19:05:48.033 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.3298 test 0.2304 metric ['0.9398']
2025-10-07 19:05:48.034 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (0.4874 --> 0.2304).Saving gestures\20251007-190517\checkpoint.pt ...
100%

Repeat 3/3


100%|██████████| 650/650 [00:08<00:00, 77.54it/s]
2025-10-07 19:08:00.208 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 1.9683 test 1.3011 metric ['0.5293']
2025-10-07 19:08:00.209 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.3011 --> 1.3011).Saving gestures\20251007-190751\checkpoint.pt ...
100%|██████████| 650/650 [00:09<00:00, 70.63it/s]
2025-10-07 19:08:10.034 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.9750 test 0.6067 metric ['0.8086']
2025-10-07 19:08:10.035 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (1.3011 --> 0.6067).Saving gestures\20251007-190751\checkpoint.pt ...
100%|██████████| 650/650 [00:09<00:00, 67.85it/s]
2025-10-07 19:08:20.174 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.4300 test 0.2579 metric ['0.9336']
2025-10-07 19:08:20.175 | INFO     | mltrainer.trainer:save_checkpoint:268 - Validation loss (0.6067 --> 0.2579).Saving gestures\20251007-190751\checkpoint.pt ...
100%

In [10]:
mlflow.end_run()

## Open MLFlow
"""
```bash
mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --host 127.0.0.1 \ 
    --port 5000 \
        
mlflow server --backend-store-uri sqlite:///3-hypertuning-rnn/mlflow.db --host 127.0.0.1 --port 5000
```
"""